In [1]:
# Importing necessary libraries
import pathlib
import time
import requests
from requests.exceptions import HTTPError
import zipfile
import tempfile
import shutil
import os
import glob
import datetime
from typing import List

# Importing Pollination Api client

from pollination_streamlit.api.client import ApiClient
from pollination_streamlit.interactors import NewJob, Recipe, Job
from queenbee.job.job import JobStatusEnum

2024-06-07 11:05:43.027 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
def submit_study(
    study_name: str, api_client: ApiClient, owner: str, project: str, epw: pathlib.Path,
        ddy: pathlib.Path, models_folder: pathlib.Path, sim_par_folder: pathlib.Path) -> Job:

    """
    This function submits a new study to Pollination.

    Args:
        study_name (str): The name of the study.
        api_client (ApiClient): The API client object.
        owner (str): The owner of the project.
        project (str): The project name.
        epw (pathlib.Path): The path to the EPW file.
        ddy (pathlib.Path): The path to the DDY file.
        models_folder (pathlib.Path): The path to the folder containing the HBJSON files.
        sim_par_folder (pathlib.Path): The path to the folder containing the simulation parameter files.

    Returns:
        Job: The job object representing the submitted study.
    """

    print(f'Creating a new study: {study_name}')
    
    # Add the recipe to the project and mention the same here. 
    recipe = Recipe('ladybug-tools', 'custom-energy-sim', '0.3.19', client=api_client)

    input_folder = pathlib.Path(models_folder)

    # create a new study
    new_study = NewJob(owner, project, recipe, client=api_client)
    new_study.name = study_name
    new_study.description = f'Annual Energy Simulation {input_folder.name}'

    # Add the epw and ddy files from diretory
    assert epw.is_file(), f'{epw} is not a valid file path.'
    assert ddy.is_file(), f'{ddy} is not a valid file path.'

    epw_path = new_study.upload_artifact(epw, target_folder='weather-data')
    ddy_path = new_study.upload_artifact(ddy, target_folder='weather-data')

    recipe_inputs = {
        'epw': epw_path,
        'ddy': ddy_path
    }

    study_inputs = []
    for model in input_folder.glob('*.hbjson'):
        inputs = dict(recipe_inputs)  # create a copy of the recipe

        # upload this model to the project
        print(f'Uploading model: {model.name}')
        uploaded_path = new_study.upload_artifact(model, target_folder=input_folder.name)
        inputs['model'] = uploaded_path
        inputs['model_id'] = model.stem  # use model name as the ID.

        # upload the corresponding sim_par.json file
        sim_par_file = sim_par_folder / f'{model.stem.replace("hbjson", "simpar")}.json'  # adjust to match your file naming convention
        if not sim_par_file.is_file():
            print(f'Warning: {sim_par_file} does not exist. Skipping this model.')
            continue
        sim_par_path = new_study.upload_artifact(sim_par_file)
        inputs['sim-par'] = sim_par_path

        study_inputs.append(inputs)

    # add the inputs to the study
    # each set of inputs create a new run
    new_study.arguments = study_inputs

    # create the study
    running_study = new_study.create()

    job_url = f'https://app.pollination.cloud/{running_study.owner}/projects/{running_study.project}/jobs/{running_study.id}'
    print(job_url)
    time.sleep(5)
    return running_study


def check_study_status(study: Job):
    """
    This function checks and prints the status of a study until it's finished.

    Args:
        study (Job): The job object representing the study.
    
    Raises:
        HTTPError: If there are more than 3 consecutive HTTP 500 errors.
    """
    
    status = study.status.status
    http_errors = 0
    while True:
        status_info = study.status
        print('\t# ------------------ #')
        print(f'\t# pending runs: {status_info.runs_pending}')
        print(f'\t# running runs: {status_info.runs_running}')
        print(f'\t# failed runs: {status_info.runs_failed}')
        print(f'\t# completed runs: {status_info.runs_completed}')
        if status in [
            JobStatusEnum.pre_processing, JobStatusEnum.running, JobStatusEnum.created,
            JobStatusEnum.unknown
        ]:
            time.sleep(15)
            try:
                study.refresh()
            except HTTPError as e:
                status_code = e.response.status_code
                print(str(e))
                if status_code == 500:
                    http_errors += 1
                    if http_errors > 3:
                        # failed for than 3 times with no success
                        raise HTTPError(e)
                    # wait for additional 15 seconds
                    time.sleep(10)
            else:
                http_errors = 0
                status = status_info.status
        else:
            # study is finished
            time.sleep(2)
            break

In [3]:
def _download_results(
    owner: str, project: str, study_id: int, download_folder: pathlib.Path,
    api_client: ApiClient):
    """
    This function downloads results from Pollination.

    Args:
    owner (str): The owner of the project.
    project (str): The project name.
    study_id (int): The ID of the study.
    download_folder (pathlib.Path): The path where results should be downloaded.
    api_client (ApiClient): The API client object.
   
   """
    per_page = 25
    page = 1
    while True:
        print(f'Downloading page {page}')
        url = f'https://api.pollination.cloud/projects/{owner}/{project}/runs'
        params = {
            'job_id': study_id,
            'status': 'Succeeded',
            'page': page,
            'per-page': per_page
        }
        response = requests.get(url, params=params, headers=api_client.headers)
        response_dict = response.json()
        runs = response_dict['resources']
        if not runs:
            break  # Exit the loop if no more runs
        temp_dir = tempfile.TemporaryDirectory()
        if temp_dir:
            temp_folder = pathlib.Path(temp_dir.name)
            for run in runs:
                run_id = run['id']
                input_id = [
                    inp['value']
                    for inp in run['status']['inputs'] if inp['name'] == 'model_id'
                ][0]
                run_folder = temp_folder.joinpath(input_id)
                sql_file = run_folder.joinpath('eplusout.sql')
                out_file = download_folder.joinpath(f'{input_id}.sql')
                print(f'downloading {input_id}.json to {out_file.as_posix()}')
                run_folder.mkdir(parents=True, exist_ok=True)
                download_folder.mkdir(parents=True, exist_ok=True)
                url = f'https://api.pollination.cloud/projects/{owner}/{project}/runs/{run_id}/outputs/sql'
                signed_url = requests.get(url, headers=api_client.headers)
                output = api_client.download_artifact(signed_url=signed_url.json())
                with zipfile.ZipFile(output) as zip_folder:
                    zip_folder.extractall(run_folder.as_posix())
                shutil.copy(sql_file.as_posix(), out_file.as_posix())
        page += 1  # Go to the next page

def download_study_results(api_client: ApiClient, study: Job, output_folder: pathlib.Path):
    """
    This function downloads the results of a study from Pollination.
    
    Args:
    api_client (ApiClient): The API client object.
    study (Job): The job object representing the study.
    output_folder (pathlib.Path): The path where results should be downloaded.
    3"""
    owner = study.owner
    project = study.project
    study_id = study.id

    _download_results(
        owner=owner, project=project, study_id=study_id, download_folder=output_folder,
        api_client=api_client
    )

In [4]:
def list_studies(api_client: ApiClient, owner: str, project: str):
    """
    This function lists all the studies in a project.

    Args:
        api_client (ApiClient): The API client object.
        owner (str): The owner of the project.
        project (str): The project name.
    """
    url = f'https://api.pollination.cloud/projects/{owner}/{project}/jobs'
    response = requests.get(url, headers=api_client.headers)
    response_dict = response.json()
    studies = response_dict['resources']
    for study in studies:
        study_name = study['spec']['name']
        print(f"Study ID: {study['id']}, Study Name: {study_name}")

# Inputs from user

In [5]:
if __name__ == '__main__':
    """
    This is the main entry point of the script. It submits a new study to Pollination,
    waits until the study is finished, and then downloads the results.
    """
    # Enter your own API Key 
    api_key = '7BA0E5BE.5B59445B8A0141E5347F5C8A'
    assert api_key is not None, 'You must provide valid Pollination API key.'

    # project owner and project name - Change these!
    owner = 'prtkwh-953'
    project = 'SA for SFH NL'

    # change this to where the study folder is
    study_folder = pathlib.Path().resolve()
    
    # Ask for iteration number and size
    itr = input("Please enter the iteration number: ")
    size = input("Please enter the size: ")
    splyT = input("Please enter the supply temperature (HT, MT, or LT): ")

    hbjson_base_dir = pathlib.Path(os.path.join(study_folder, "3. HBjson"))
    simpar_base_dir = pathlib.Path(os.path.join(study_folder, "4. Sim_par"))
    sqlfiles_base_dir = pathlib.Path(os.path.join(study_folder, "5. Sql_files"))
    
    hbjson_subdirs = glob.glob(os.path.join(hbjson_base_dir, f'*_itr_{itr}_size_{size}_{splyT}'))
    simpar_subdirs = glob.glob(os.path.join(simpar_base_dir, f'*_itr_{itr}_size_{size}_{splyT}'))
    sqlfiles_subdirs = glob.glob(os.path.join(sqlfiles_base_dir, f'*_itr_{itr}_size_{size}_{splyT}'))

    # Select the first matching subdirectory for each base directory
    input_folder = pathlib.Path(hbjson_subdirs[0]) if hbjson_subdirs else None
    sim_path = pathlib.Path(simpar_subdirs[0]) if simpar_subdirs else None
    results_folder = pathlib.Path(sqlfiles_subdirs[0]) if sqlfiles_subdirs else None

    # Ensure none of the paths are None
    assert input_folder is not None, 'Input folder not found'
    assert sim_path is not None, 'Sim path folder not found'
    assert results_folder is not None, 'Results folder not found'

    epw = pathlib.Path(os.path.join(study_folder, 'PW_NEN5060_2021_EPW.epw'))
    ddy = pathlib.Path(os.path.join(study_folder, 'NEN5060_min10_Hddy.ddy'))
    
    # Construct study name based on date, iteration number and size
    name = f'{datetime.date.today()}_th_itr_{itr}_size_{size}_{splyT}'
    
    api_client = ApiClient(api_token=api_key)

    study = submit_study(name, api_client, owner, project, epw, ddy, input_folder, sim_path)
    
    # wait until the study is finished
    check_study_status(study=study)
    
    download_study_results(
        api_client=api_client, study=study, output_folder=results_folder
    )

Please enter the iteration number:  1
Please enter the size:  4
Please enter the supply temperature (HT, MT, or LT):  HT


Creating a new study: 2024-06-07_th_itr_1_size_4_HT
Uploading model: Th_hbjson_sample_1.hbjson
Uploading model: Th_hbjson_sample_2.hbjson
Uploading model: Th_hbjson_sample_3.hbjson
Uploading model: Th_hbjson_sample_4.hbjson
https://app.pollination.cloud/prtkwh-953/projects/SA for SFH NL/jobs/ec1e7bde-6c5a-4eed-bdb1-c12f9ae1c0ea
	# ------------------ #
	# pending runs: 0
	# running runs: 0
	# failed runs: 0
	# completed runs: 0
	# ------------------ #
	# pending runs: 0
	# running runs: 4
	# failed runs: 0
	# completed runs: 0
	# ------------------ #
	# pending runs: 0
	# running runs: 4
	# failed runs: 0
	# completed runs: 0
	# ------------------ #
	# pending runs: 0
	# running runs: 4
	# failed runs: 0
	# completed runs: 0
	# ------------------ #
	# pending runs: 0
	# running runs: 4
	# failed runs: 0
	# completed runs: 0
	# ------------------ #
	# pending runs: 0
	# running runs: 4
	# failed runs: 0
	# completed runs: 0
	# ------------------ #
	# pending runs: 0
	# running runs: 0
	#

downloading Th_hbjson_sample_847.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_847.sql
downloading Th_hbjson_sample_846.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_846.sql
downloading Th_hbjson_sample_845.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_845.sql
downloading Th_hbjson_sample_844.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses

downloading Th_hbjson_sample_82.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_82.sql
downloading Th_hbjson_sample_819.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_819.sql
downloading Th_hbjson_sample_818.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_818.sql
downloading Th_hbjson_sample_817.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/1

downloading Th_hbjson_sample_793.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_793.sql
downloading Th_hbjson_sample_792.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_792.sql
downloading Th_hbjson_sample_791.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_791.sql
downloading Th_hbjson_sample_790.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses

...

downloading Th_hbjson_sample_1125.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_1125.sql
downloading Th_hbjson_sample_1124.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_1124.sql
downloading Th_hbjson_sample_1123.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_1123.sql
downloading Th_hbjson_sample_1122.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced

downloading Th_hbjson_sample_1099.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_1099.sql
downloading Th_hbjson_sample_1098.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_1098.sql
downloading Th_hbjson_sample_1097.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_1097.sql
downloading Th_hbjson_sample_1096.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced

downloading Th_hbjson_sample_1071.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_1071.sql
downloading Th_hbjson_sample_1070.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_1070.sql
downloading Th_hbjson_sample_107.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_107.sql
downloading Th_hbjson_sample_1069.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced h

downloading Th_hbjson_sample_1044.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_1044.sql
downloading Th_hbjson_sample_1043.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_1043.sql
downloading Th_hbjson_sample_1042.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_1042.sql
downloading Th_hbjson_sample_1041.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced

downloading Th_hbjson_sample_1017.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_1017.sql
downloading Th_hbjson_sample_1016.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_1016.sql
downloading Th_hbjson_sample_1015.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced houses/10.3 updated_cr_logic_supplyT/6. Sql_files/2024-04-23_sqlfiles_itr_1_size_1300_LT/Th_hbjson_sample_1015.sql
downloading Th_hbjson_sample_1014.json to C:/Users/prateekwahi/OneDrive - Delft University of Technology/PhD/7. Reports/3. Papers/3. paper 3/simulations/6.Terraced

# Use this to download specifc study

In [ ]:
api_client = ApiClient(api_token=api_key)
 # List all studies
list_studies(api_client, owner, project)

In [ ]:
# Download a specific study
study_id = input("Please enter the study_ID: ")
output_folder = pathlib.Path(input('Your Output Folder'))
_download_results(
        owner=owner, project=project, study_id=study_id, download_folder=output_folder,
        api_client=api_client
    )